1. Using prompttemplates and output parsers
2. Using LCEL
3. Debugging using Langsmith
4. Deploying using LangServe

Llama3, Gemma2, mistral -- Groq

LPU - Language processing Unit better in compute density and memory bandwidth


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11bf301f0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11bf30d60>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage #Instruction to the LLM model
msgs = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, how are you?")
]
result = model.invoke(msgs)


In [9]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(result)

'Bonjour, comment allez-vous ? \n\n\nThis is the most formal way to say it.  Here are some other options:\n\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends) \n'

In [10]:
# Chain
chain = model | output_parser
chain.invoke(msgs)

"Bonjour, comment allez-vous ? \n\n\nYou can also use:\n\n* **Salut, ça va ?** (Informal)\n\n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [14]:
# Prompt Template
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("user", "{text}")
     ]
)
result = prompt_template.invoke({"language": "German", "text": "Hello, how are you?"})
result.to_messages()


[SystemMessage(content='Translate the following into German:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [16]:
chain = prompt_template | model | output_parser
chain.invoke({"language": "German", "text": "Hello, how are you?"})

'Hallo, wie geht es dir? \n\n\nYou can also say:\n\n* **Hallo, wie geht es Ihnen?** (formal)\n'